<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>ChatGPT and Friends</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import openai
from openai import OpenAI

import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: b44900a26f10de8fbaf559b307e69185828b77b4

termcolor : 2.4.0
json      : 2.0.9
watermark : 2.4.3
pandas    : 2.2.3
openai    : 1.30.5
tqdm      : 4.66.4
matplotlib: 3.8.0
numpy     : 1.26.4



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is always to load up the API key from the local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

We start by getting a list of supported models.

In [5]:
model_list = client.models.list().data

In total we have 38 models

In [6]:
len(model_list)

38

Along with some information about each model...

In [7]:
model_list[:3]

[Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system')]

But let's just get a list of model names

In [8]:
print("\n".join(sorted([model.id for model in model_list])))

babbage-002
chatgpt-4o-latest
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4o
gpt-4o-2024-05-13
gpt-4o-2024-08-06
gpt-4o-audio-preview
gpt-4o-audio-preview-2024-10-01
gpt-4o-mini
gpt-4o-mini-2024-07-18
gpt-4o-realtime-preview
gpt-4o-realtime-preview-2024-10-01
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-4o`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [9]:
%%time
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

CPU times: user 9.51 ms, sys: 2.51 ms, total: 12 ms
Wall time: 1.65 s


Which produces a response object

In [10]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a simple object

In [11]:
pprint(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's most well-known weakness is Kryptonite, a radioactive substance from his home planet, Krypton. Exposure to Kryptonite can weaken Superman, stripping him of his powers and making him vulnerable to harm. There are different varieties of Kryptonite, with green Kryptonite being the most common and harmful to him. Additionally, Superman is susceptible to magic and can be affected by red solar radiation, which can deplete his powers.", role='assistant', function_call=None, tool_calls=None, refusal=None))


The model answer can be found in the "message" named tuple inside the "choices" list

In [11]:
print(response.choices[0].message.content)

Superman's primary weakness is kryptonite, a mineral from his home planet of Krypton. When exposed to kryptonite, Superman loses his powers and becomes physically weakened, which can put him in danger. There are different variations of kryptonite, with green kryptonite being the most common and most harmful to him. Additionally, Superman is also vulnerable to magic and the effects of a red sun, which can strip him of his powers.


To request multiple answers, we must include the `n` parameter with the number of answers we want

In [12]:
%%time
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

CPU times: user 10.9 ms, sys: 2.35 ms, total: 13.2 ms
Wall time: 9.86 s


And we can access each of the answers individually int he choices list

In [13]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
Kryptonite is a fictional substance from the Superman comic books, part of the DC Comics universe. It's famously known for its debilitating effects on Superman and other Kryptonians. Over the years, various types of Kryptonite have been introduced, each with different properties and effects:

1. **Green Kryptonite**: The most well-known form, Green Kryptonite weakens Kryptonians and prolonged exposure can be fatal.

2. **Red Kryptonite**: This variety causes unpredictable and usually bizarre changes in Superman. The effects are temporary but can vary greatly from one exposure to another.

3. **Blue Kryptonite**: Typically affects Bizarro, the imperfect clone of Superman, in the same way Green Kryptonite affects Superman. It can also sometimes have calming effects on Kryptonians.

4. **Gold Kryptonite**: Removes a Kryptonian's superpowers permanently.

5. **White Kryptonite**: Kills all plant life, including extraterrestrial varieties.

6. **Black Kryptonite**: Has the ability

In [14]:
response.usage

CompletionUsage(completion_tokens=1197, prompt_tokens=16, total_tokens=1213, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0})

# Temperature

In [ ]:
%%time
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "Tell me a fun fact"},
    ],
    temperature=1.9
)

In [ ]:
print(response.choices[0].message.content)

In [24]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=0
)

CPU times: user 8.24 ms, sys: 2.15 ms, total: 10.4 ms
Wall time: 3.88 s


In [25]:
print(response.choices[0].message.content)

Once upon a time, in a small village nestled in the mountains, there lived a young girl named Lily. Lily was known for her kindness and generosity, always willing to help those in need.

One day, a terrible storm hit the village, causing widespread damage and leaving many families homeless. Lily knew she had to do something to help. She gathered her friends and together they started a fundraiser to rebuild the homes that were destroyed.

Through hard work and determination, Lily and her friends were able to raise enough money to rebuild every home in the village. The villagers were overjoyed and grateful for Lily's selfless act of kindness.

From that day on, Lily became a hero in the village, known for her compassion and willingness to help others in need. She showed everyone that even the smallest act of kindness can make a big difference in the world. And so, the village lived happily ever after, thanks to the kindness of one young girl named Lily.


## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [26]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

Because of this last-minute change, we can't do everything for the client's project.


# Formatted output

In [ ]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

In [29]:
pprint(prompt)

('return a recipe for blueberry pancakes.\n'
 '        Provide your response as a JSON object with the following schema:\n'
 '        {"dish": "blueberry pancakes", "ingredients": ["", "", ...],\n'
 '        "instructions": ["", "", ... ]}')


In [28]:
response = client.chat.completions.create(
          model = "gpt-4o",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 8.31 ms, sys: 2.22 ms, total: 10.5 ms
Wall time: 5.23 s


In [30]:
json_output = response.choices[0].message.content

In [31]:
print(json_output)

```json
{
    "dish": "blueberry pancakes",
    "ingredients": [
        "1 cup all-purpose flour",
        "1 tablespoon sugar",
        "1 teaspoon baking powder",
        "1/2 teaspoon baking soda",
        "1/4 teaspoon salt",
        "1 cup buttermilk",
        "1 large egg",
        "2 tablespoons unsalted butter, melted",
        "1 cup fresh blueberries",
        "Extra butter or oil for cooking"
    ],
    "instructions": [
        "In a large bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.",
        "In another bowl, whisk together the buttermilk, egg, and melted butter.",
        "Pour the wet ingredients into the dry ingredients and stir until just combined. The batter will be lumpy; do not overmix.",
        "Gently fold in the blueberries.",
        "Heat a non-stick skillet or griddle over medium heat and lightly coat with butter or oil.",
        "Pour 1/4 cup of batter for each pancake onto the skillet.",
        "Cook until bubbles form on

In [32]:
output = json.loads(json_output[7:-3])

In [33]:
output["ingredients"]

['1 cup all-purpose flour',
 '1 tablespoon sugar',
 '1 teaspoon baking powder',
 '1/2 teaspoon baking soda',
 '1/4 teaspoon salt',
 '1 cup buttermilk',
 '1 large egg',
 '2 tablespoons unsalted butter, melted',
 '1 cup fresh blueberries',
 'Extra butter or oil for cooking']

In [34]:
output["instructions"]

['In a large bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.',
 'In another bowl, whisk together the buttermilk, egg, and melted butter.',
 'Pour the wet ingredients into the dry ingredients and stir until just combined. The batter will be lumpy; do not overmix.',
 'Gently fold in the blueberries.',
 'Heat a non-stick skillet or griddle over medium heat and lightly coat with butter or oil.',
 'Pour 1/4 cup of batter for each pancake onto the skillet.',
 'Cook until bubbles form on the surface of the pancake and the edges look set, about 2-3 minutes.',
 'Flip and cook the other side until golden brown, about 2 minutes more.',
 'Repeat with the remaining batter, adding more butter or oil to the skillet as needed.',
 'Serve warm with your choice of syrup or toppings.']

# Translation

In [35]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [36]:
response.choices[0].message.content

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [37]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [38]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [39]:
print(response.choices[0].message.content)

```csv
Fruit,Color,Taste
Neoskizzles,Purple,Candy
Loheckles,Grayish Blue,Tart
Pounits,Bright Green,Savory
Loopnovas,Neon Pink,Cotton Candy
Glowls,Pale Orange,Sour and Bitter
```


In [40]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [41]:
print(response.choices[0].message.content)

Pounits are a bright green fruit found on the planet Goocrux, and they are more savory than sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>